# Wrapping a template library

A template library is a library where there are only template classes that can be instantiated.
Wrapping such libraries therefore requires **AutoWIG** to be able to consider various *C++* template classes instantiations during the `Parse` step.
It is therefore required to install the `pyclanglite` `parser`.

The **Standard Template Library (STL)** library is a *C++* library that provides a set of common *C++* template classes such as containers and associative arrays.
These classes can be used with any built-in or user-defined type that supports some elementary operations (e.g. copying, assignment).
It is divided in four components called algorithms, containers, functional and iterators.
**STL** containers (e.g. `std::vector`, `std::set`) are used in many *C++* libraries.
In such a case, it does not seem relevant that every wrapped *C++* library contains wrappers for usual **STL** containers (e.g. `std::vector< double >`, `std::set< int >`).
We therefore proposed *Python* bindings for sequence containers (i.e. `pair`, `array`, `vector`, `deque`, `forward_list` and `list` of the `std` namespace) and associative containers (`set`, `multiset`, `map`, `multimap`, `unordered_set`, `unordered_multiset`, `unordered_map` and `unordered_multimap` of the `std` namespace).
These template instantiations are done for *C++* fundamental types (`bool`, `signed char`, `unsigned char`, `char`, `wchar_t`, `int` (with sign modifiers `signed` and `signed` combined or not with size modifiers `short`, `long` and `long long`), `float`, `double`, `long double`) and strings (`string`, `wstring` of the `std` namespace).
For ordered associative containers both `std::less` and `std::greater` comparators are used.
We here only illustrate the procedure on the `std::vector` template class.
For the complete procedure refers to the `AutoWIG.py` file situed at the root of the **PySTL** [repository](https://github.com/StatisKit/PySTL).

Then, to install and compile the *C++* library we use available **Conda** recipes.

In [1]:
!conda build -q ../git/STL/conda/libstatiskit_stl -c statiskit
!conda install -y -q libstatiskit_stl --use-local -c statiskit




####################################################################################
Source and build intermediates have been left in /home/main/miniconda/conda-bld.
There are currently 2 accumulated.
To remove them, you can run the ```conda build purge``` command


Package plan for installation in environment /home/main/miniconda:

The following NEW packages will be INSTALLED:

    libstatiskit_stl: 1.0.0-0       statiskit  

The following packages will be UPDATED:

    conda:            4.3.23-py36_0 conda-forge --> 4.3.25-py36_0

The following packages will be SUPERSEDED by a higher-priority channel:

    conda-env:        2.6.0-0       conda-forge --> 2.6.0-0      



As presented below, in order to wrap a template library, the user needs to write headers containing aliases for desired template class instantiations.

In [2]:
!pygmentize ../git/STL/src/cpp/STL.h

#ifndef STATISKIT_STL_H
#define STATISKIT_STL_H

#include <vector>
#include <set>
#include <unordered_set>
#include <string>

#if defined WIN32 || defined _WIN32 || defined __CYGWIN__
    #ifdef LIBSTATISKIT_STL
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllexport))
        #else
            #define STATISKIT_STL_API __declspec(dllexport)
        #endif
    #else
        #ifdef __GNUC__
            #define STATISKIT_STL_API __attribute__ ((dllimport))
        #else
            #define STATISKIT_STL_API __declspec(dllimport)
        #endif
    #endif
#else
    #if __GNUC__ >= 4
        #define STATISKIT_STL_API __attribute__ ((visibility ("default")))
    #else
        #define STATISKIT_STL_API
    #endif
#endif

namespace statiskit
{
    typedef unsigned long int Index;
    typedef std::set< Index > Indices;

    namespace stl
    {

        template<class T>
            class Generator
            {
                public:
                    Generat

Once these preliminaries done, we can proceed to the actual generation of wrappers for the **PySTL** library.
For this, we import **AutoWIG** and create an empty Abstract Semantic Graph (ASG).

We need then to install the *C++* headers. This is done using the `cpp` target in **SCons**.

In [3]:
!scons cpp -C ../git/STL

scons: Entering directory `/home/main/repository/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home/main/miniconda/include/statiskit/stl/STL.h"
g++ -o build/src/cpp/STL.os -c -x c++ -std=c++11 -fmax-errors=0 -Wl,--no-undefined -fvisibility=hidden -fdiagnostics-color=always -fPIC -DLIBSTATISKIT_STL -I/home/main/miniconda/include build/src/cpp/STL.cpp
g++ -o /home/main/miniconda/lib/libstatiskit_stl.so -shared build/src/cpp/STL.os -L/home/main/miniconda/lib
scons: done building targets.


Once the headers habe been installed in the system, we parse headers with relevant compilation flags.

In [4]:
!scons autowig -c -C ../git/STL
!scons autowig -C ../git/STL

scons: Entering directory `/home/main/repository/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Cleaning targets ...
Removed /home/main/miniconda/include/statiskit/stl/STL.h
scons: done cleaning targets.
scons: Entering directory `/home/main/repository/share/git/STL'
scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/src/cpp/STL.h" as "/home/main/miniconda/include/statiskit/stl/STL.h"
Install file: "build/src/cpp/AutoWIG.py" as "/home/main/miniconda/lib/python3.6/site-packages/autowig/site/controller/statiskit_stl.py"
autowig: Generating Boost.Python interface ...
scons: *** [/home/main/miniconda/lib/python3.6/site-packages/autowig/site/ASG/statiskit_stl.pkl] NameError : name 'basestring' is not defined
Traceback (most recent call last):
  File "/home/main/miniconda/lib/python3.6/site-packages/SCons/Action.py", line 1192, in execute
    result = self.execfunction(tar

Here is the list of the generated wrappers (untracked files).

In [5]:
!cd ../git/STL && git status

HEAD detached at c33f5e9
Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git checkout -- <file>..." to discard changes in working directory)

	modified:   src/cpp/STL.cpp
	modified:   src/cpp/STL.h

no changes added to commit (use "git add" and/or "git commit -a")


And here, we present the wrappers for the `std::vector< int >` class.

In [6]:
!pygmentize ../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp

Error: cannot read infile: [Errno 2] No such file or directory: '../git/STL/src/py/wrapper/wrapper_6b9ae5eac40858c9a0f5e6e21c15d1d3.cpp'


Once the wrappers are written on disk, we need to compile and install the *Python* bindings.

In [7]:
!conda build ../git/STL/bin/conda/python-statiskit_stl -c statiskit -c conda-forge
!conda install -y python-statiskit_stl --use-local -c statiskit --force

BUILD START: python-statiskit_stl-3.3.1-py36_0
    (actual version deferred until further download or env creation)

The following NEW packages will be INSTALLED:

    ca-certificates:  2017.7.27.1-0      conda-forge
    certifi:          2017.7.27.1-py36_0 conda-forge
    coverage:         4.4.1-py36_0       conda-forge
    icu:              58.1-1             conda-forge
    libboost:         1.61.0-py36_0      statiskit  
    libdev:           1.0.0-py36_0       statiskit  
    librun:           1.0.0-0            statiskit  
    libstatiskit_stl: 1.0.0-0            statiskit  
    ncurses:          5.9-10             conda-forge
    nose:             1.3.7-py36_2       conda-forge
    openssl:          1.0.2l-0           conda-forge
    path.py:          10.3.1-py36_0      conda-forge
    pip:              9.0.1-py36_0       conda-forge
    python:           3.6.2-0            conda-forge
    python-dev:       1.0.0-py36_0       statiskit  
    python-scons:     3.0.0-py36_0       

scons: done building targets.

set +ve
+ set +ve
INFO:conda_build.build:Packaging python-statiskit_stl-3.3.1-py36_0
fatal: Not a git repository: /home/main/miniconda/conda-bld/python-statiskit_stl_1505308958619/work/../../../.git/modules/share/git/STL
fatal: Not a git repository: /home/main/miniconda/conda-bld/python-statiskit_stl_1505308958619/work/../../../.git/modules/share/git/STL
compiling .pyc files...
number of files: 11
Fixing permissions
patchelf: file: /home/main/miniconda/conda-bld/python-statiskit_stl_1505308958619/_b_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pl/lib/python3.6/site-packages/statiskit/stl/__stl.so
    setting rpath to: $ORIGIN/../../../..
Fixing permissions
/home/main/miniconda/conda-bld/linux-64/python-statiskit_stl-3.3.1-py36_0.tar.bz2
updating index in: /home/main/miniconda/conda-bld
updating index in: /home/main/min

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [8]:
from statiskit.stl import VectorInt
v = VectorInt()
v.push_back(-1)
v.push_back(0)
v.push_back(1)
v

ModuleNotFoundError: No module named 'set'

In [ ]:
list(v)

In [ ]:
v[0]

In [ ]:
v[0] = -2
v[0]

In [ ]:
VectorInt([0, 1])

Here is a report concerning objects wrapped using this notebook.

In [ ]:
import fp17
import os
import pickle
with open(os.path.join(os.environ['SCONSFLAGS'].lstrip('--site-dir='),
                       '..',
                       '..',
                       'autowig',
                       'site',
                       'ASG',
                       'statiskit_stl.pkl'), 'r') as filehandler:
    asg = pickle.load(filehandler)
fp17.report(asg)